In [1]:
import sys
sys.path.append('/home/shuirh/project/research/invariant_learning')

In [2]:
import importlib
import os
import numpy as np

In [3]:
if 'trainer' in globals():
    importlib.reload(trainer)
else:
    import modules.trainer as trainer

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
class MNIST_DS(Dataset):
    def __init__(self, vd):
        self.vd = vd
    def __len__(self):
        return len(self.vd)
    def __getitem__(self, idx):
        x,y = self.vd[idx]
        return {
            'x': x,
            'labels': y
        }
tr_ds = MNIST_DS(training_data)
te_ds = MNIST_DS(test_data)

In [6]:
# Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x, labels = None):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        outputs = {'logits': logits}
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
            outputs['loss'] = loss
        return outputs

model = NeuralNetwork()

In [7]:
def compute_accuracy(all_logits, all_labels):
    """
    Metric should start with `eval_`
    """
    all_preds = np.argmax(all_logits, axis = -1)
    num_samples = len(all_preds)
    acc = (all_preds == all_labels).sum() / num_samples
    metrics = {
        'eval_acc': acc
    }
    return metrics
args1 = trainer.SimpleTrainingArguments(
    output_dir = './trainer_output',
    eval_epochs = 1,
    num_train_epochs = 3,
    learning_rate = 1e-2,
    per_device_train_batch_size = 64
)
trainer1 = trainer.BasicTrainer(
    model,
    args = args1,
    train_dataset = tr_ds,
    eval_dataset = te_ds,
    compute_metrics = compute_accuracy
)

In [9]:
trainer1.train()

***** Running training *****
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Total optimization steps = 2814
  0%|                                                  | 0/2814 [00:00<?, ?it/s]/home/shuirh/lib/anaconda3/envs/st/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 33%|█████████████▎                          | 937/2814 [00:14<00:26, 71.10it/s]***** Running evaluation *****
  Num examples = 1250
  Batch size = 8
{'eval_acc': 0.9621, 'step': 938, 'epoch': 1.0}
Saving model checkpoint to ./trainer_output/checkpoint-938
 34%|█████████████▍                          | 945/2814 [00:19<05:54,  5.27it/s]

{'eval_acc': 0.9621}


 67%|█████████████████████████▉             | 1874/2814 [00:33<00:14, 66.23it/s]***** Running evaluation *****
  Num examples = 1250
  Batch size = 8
{'eval_acc': 0.9737, 'step': 1876, 'epoch': 2.0}
Saving model checkpoint to ./trainer_output/checkpoint-1876
 67%|██████████████████████████▏            | 1887/2814 [00:38<02:44,  5.64it/s]

{'eval_acc': 0.9737}


100%|██████████████████████████████████████▉| 2811/2814 [00:51<00:00, 72.01it/s]***** Running evaluation *****
  Num examples = 1250
  Batch size = 8
{'eval_acc': 0.9832, 'step': 2814, 'epoch': 3.0}
Saving model checkpoint to ./trainer_output/checkpoint-2814
100%|███████████████████████████████████████| 2814/2814 [00:57<00:00, 49.12it/s]

{'eval_acc': 0.9832}


In [8]:
args2 = trainer.SimpleTrainingArguments(
    output_dir = './trainer_output',
    eval_steps = 1000,
    max_steps = 2000,
    learning_rate = 1e-2,
    per_device_train_batch_size = 64
)
trainer2 = trainer.BasicTrainer(
    model,
    args = args2,
    train_dataset = tr_ds,
    eval_dataset = te_ds,
    compute_metrics = compute_accuracy
)

In [9]:
trainer2.train()

***** Running training *****
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Total optimization steps = 2000
  0%|                                                  | 0/2000 [00:00<?, ?it/s]/home/shuirh/lib/anaconda3/envs/st/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 50%|███████████████████▉                    | 998/2000 [00:15<00:13, 71.83it/s]{'loss': 0.09435329080559313, 'learning_rate': 0.0005, 'step': 1000, 'epoch': 1.07}
***** Running evaluation *****
  Num examples = 1250
  Batch size = 8
{'eval_acc': 0.9709, 'step': 1000, 'epoch': 1.07}
Saving model checkpoint to ./trainer_output/checkpoint-1000
100%|██████████████████████████████████████▉| 1998/2000 [00:34<00:00, 67.54it/s]{'l

In [10]:
args3 = trainer.SimpleTrainingArguments(
    eval_steps = 1000,
    max_steps = 2000,
    learning_rate = 1e-2,
    per_device_train_batch_size = 64
)
trainer3 = trainer.BasicTrainer(
    model,
    args = args3,
    train_dataset = tr_ds,
    eval_dataset = te_ds,
    compute_metrics = compute_accuracy
)

In [11]:
trainer3.train()

***** Running training *****
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Total optimization steps = 2000
 50%|███████████████████▉                    | 999/2000 [00:13<00:12, 77.04it/s]{'loss': 0.04229187134507811, 'learning_rate': 0.0005, 'step': 1000, 'epoch': 1.07}
***** Running evaluation *****
  Num examples = 1250
  Batch size = 8
{'eval_acc': 0.9781, 'step': 1000, 'epoch': 1.07}
100%|██████████████████████████████████████▉| 1996/2000 [00:33<00:00, 74.68it/s]{'loss': 0.017086673446465285, 'learning_rate': 0.0, 'step': 2000, 'epoch': 2.13}
***** Running evaluation *****
  Num examples = 1250
  Batch size = 8
{'eval_acc': 0.9845, 'step': 2000, 'epoch': 2.13}
100%|███████████████████████████████████████| 2000/2000 [00:39<00:00, 50.61it/s]


In [12]:
from transformers import BertForSequenceClassification

In [15]:
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_class = 21)

TypeError: __init__() got an unexpected keyword argument 'num_class'

In [44]:
trainer3.lr_scheduler.get_last_lr()

[0.0]

## Collator

In [24]:
import modules
from transformers.data.data_collator import default_data_collator
from torch.utils.data import DataLoader

In [18]:
import pickle

In [31]:
with open('/home/shuirh/project/research/invariant_learning/rnp/cache/cail_transformer_ds.pkl', 'rb') as f:
    train,dev,test = pickle.load(f)

In [32]:
train.set_label_name('labels_charge')

In [25]:
train_dl = DataLoader(train, batch_size = 8, collate_fn = default_data_collator)

In [33]:
for k,v in train[2].items():
    print(f"{k}: {len(v) if isinstance(v, list) else 0}")

input_ids: 518
attention_mask: 518
token_type_ids: 518
labels: 0


In [34]:
type(train[0]['input_ids'])

list

In [42]:
(torch.LongTensor([1,2,3]) == torch.LongTensor([1,2,0])).float().mean()

tensor(0.6667)